# SubtleGenius v1 - ARC Prize 2025 Submission System
## Lean Agile Solo Vibe: Coder-LLM Collaborative AI Fusion

**Architecture**: 6-Cell Modular Design
**Philosophy**: Production-First, Token-Efficient, Never-Crash
**Target**: Valid submission.json on first run, iterate to 85%+ accuracy

---

## Cell Architecture
1. **Configuration & Imports** - Zero-dependency foundation
2. **Submission Validator** - Build BEFORE solving (Validation > Innovation)
3. **Safe Defaults & Fallbacks** - Never crash (Valid 5% > Crashing 95%)
4. **Submission Generator** - Wire to solver with production orchestration
5. **Solver Logic** - Your AGI genius goes here (expand iteratively)
6. **Execution Pipeline** - Load → Solve → Validate → Save

**Based on**: 48 hours of Ryan & Claude breakthrough insights

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: CONFIGURATION & IMPORTS
# Zero cross-cell dependency issues - Single source of truth
# ═══════════════════════════════════════════════════════════════════════════════

import json
import numpy as np
import os
import time
import gc
from typing import Dict, List, Any, Tuple, Optional
from pathlib import Path
from datetime import datetime
import traceback

print("🐋 SubtleGenius v1 - ARC Prize 2025")
print(f"⏰ Initialized: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION (Meta-Optimized from 48hr Insights)
# ═══════════════════════════════════════════════════════════════════════════════

class ARC2025Config:
    """Single source of truth for all configuration"""
    
    # Environment detection (auto-detect Kaggle vs local)
    IS_KAGGLE = os.path.exists('/kaggle/input')
    
    if IS_KAGGLE:
        # Kaggle paths
        INPUT_PATH = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'
        TRAIN_PATH = '/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json'
        OUTPUT_PATH = '/kaggle/working/submission.json'
        LOG_PATH = '/kaggle/working/solver_log.jsonl'
    else:
        # Local development paths
        INPUT_PATH = 'data/arc-agi_test_challenges.json'
        TRAIN_PATH = 'data/arc-agi_training_challenges.json'
        OUTPUT_PATH = 'submission.json'
        LOG_PATH = 'solver_log.jsonl'
    
    # Time budget management (95% rule - never terminate early!)
    TOTAL_TIME_LIMIT = 12 * 3600  # 12 hours for ARC 2025
    SAFETY_BUFFER = 0.05  # Use 95% of time (Insight #4)
    EFFECTIVE_TIME = TOTAL_TIME_LIMIT * (1 - SAFETY_BUFFER)
    
    # Submission requirements (from official ARC Prize 2025 docs)
    REQUIRED_ATTEMPTS = 2  # attempt_1 and attempt_2
    GRID_VALUE_RANGE = (0, 9)  # Valid pixel values
    
    # Production settings
    ENABLE_LOGGING = True
    ENABLE_VALIDATION = True
    ENABLE_FALLBACKS = True
    VERBOSE = True
    
    # Dynamic difficulty multipliers (Insight #4: Dynamic Time Budgeting)
    DIFFICULTY_MULTIPLIERS = {
        'easy': 0.5,
        'medium': 1.0,
        'hard': 2.0,
        'elite': 3.0
    }

config = ARC2025Config()

# ═══════════════════════════════════════════════════════════════════════════════
# GLOBAL TIMER (95% Rule Enforcement)
# ═══════════════════════════════════════════════════════════════════════════════

class GlobalTimer:
    """Track time budget and enforce 95% rule"""
    
    def __init__(self):
        self.start_time = time.time()
    
    def elapsed(self):
        """Time elapsed in seconds"""
        return time.time() - self.start_time
    
    def remaining(self):
        """Time remaining in seconds"""
        return config.EFFECTIVE_TIME - self.elapsed()
    
    def should_stop(self):
        """Check if we should wrap up (10min buffer)"""
        return self.remaining() < 600
    
    def progress_pct(self):
        """Percentage of time budget used"""
        return (self.elapsed() / config.EFFECTIVE_TIME) * 100

timer = GlobalTimer()

print(f"✅ Configuration loaded")
print(f"   Environment: {'Kaggle' if config.IS_KAGGLE else 'Local'}")
print(f"   Time budget: {config.EFFECTIVE_TIME/3600:.1f} hours")
print(f"   Output: {config.OUTPUT_PATH}")
print(f"   95% rule: {config.SAFETY_BUFFER*100}% safety buffer\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: SUBMISSION VALIDATOR
# "Validation > Innovation" - Build BEFORE solving code
# Prevents wasting daily submissions on format errors
# ═══════════════════════════════════════════════════════════════════════════════

class SubmissionValidator:
    """
    Production-grade validator for ARC Prize 2025 submissions.
    
    Based on lessons from failed submission:
    - ❌ List instead of dict
    - ❌ Missing attempt_1/attempt_2 keys
    - ❌ task_id as internal key (should be top-level)
    - ❌ Single prediction (need 2 attempts)
    - ❌ Not handling multiple test outputs
    """
    
    @staticmethod
    def validate_grid(grid: List[List[int]], context: str = "") -> Tuple[bool, str]:
        """Validate a single grid structure"""
        
        # Must be a list
        if not isinstance(grid, list):
            return False, f"{context}: Grid must be list, got {type(grid)}"
        
        # Cannot be empty
        if len(grid) == 0:
            return False, f"{context}: Grid cannot be empty"
        
        # Must be 2D list
        if not all(isinstance(row, list) for row in grid):
            return False, f"{context}: Grid must be 2D list"
        
        # No ragged arrays (all rows same length)
        if len(grid) > 0:
            row_len = len(grid[0])
            if not all(len(row) == row_len for row in grid):
                return False, f"{context}: Ragged array detected"
        
        # All values must be integers 0-9
        for i, row in enumerate(grid):
            for j, val in enumerate(row):
                if not isinstance(val, int):
                    return False, f"{context}[{i}][{j}]: Must be int, got {type(val)}"
                if val < config.GRID_VALUE_RANGE[0] or val > config.GRID_VALUE_RANGE[1]:
                    return False, f"{context}[{i}][{j}]: Value {val} not in range 0-9"
        
        return True, "Valid grid"
    
    @staticmethod
    def validate_prediction(prediction: Dict, context: str = "") -> Tuple[bool, str]:
        """Validate a single prediction dict (one test output)"""
        
        # Must be a dictionary
        if not isinstance(prediction, dict):
            return False, f"{context}: Prediction must be dict, got {type(prediction)}"
        
        # Must have both attempts
        if "attempt_1" not in prediction:
            return False, f"{context}: Missing 'attempt_1' key"
        if "attempt_2" not in prediction:
            return False, f"{context}: Missing 'attempt_2' key"
        
        # Validate both attempt grids
        for attempt_key in ["attempt_1", "attempt_2"]:
            grid = prediction[attempt_key]
            is_valid, msg = SubmissionValidator.validate_grid(
                grid, 
                context=f"{context}[{attempt_key}]"
            )
            if not is_valid:
                return False, msg
        
        return True, "Valid prediction"
    
    @staticmethod
    def validate_task_predictions(task_id: str, 
                                  predictions: List[Dict],
                                  expected_count: int) -> Tuple[bool, str]:
        """Validate all predictions for a single task"""
        
        # Must be a list
        if not isinstance(predictions, list):
            return False, f"Task {task_id}: predictions must be list, got {type(predictions)}"
        
        # Must match expected number of test outputs
        if len(predictions) != expected_count:
            return False, f"Task {task_id}: expected {expected_count} predictions, got {len(predictions)}"
        
        # Validate each prediction
        for idx, prediction in enumerate(predictions):
            is_valid, msg = SubmissionValidator.validate_prediction(
                prediction,
                context=f"Task {task_id}[{idx}]"
            )
            if not is_valid:
                return False, msg
        
        return True, "Valid task predictions"
    
    @staticmethod
    def validate_submission(submission: Dict, 
                           test_challenges: Dict) -> Tuple[bool, str]:
        """
        Validate complete submission.json structure.
        This is the MASTER validation function.
        """
        
        print("\n" + "="*70)
        print("🔍 VALIDATING SUBMISSION")
        print("="*70)
        
        # 1. Must be a dictionary (NOT a list!)
        if not isinstance(submission, dict):
            return False, f"❌ Submission must be DICT, got {type(submission)}"
        
        print(f"✅ Structure: Dictionary (not list)")
        
        # 2. Must have ALL task IDs from test set
        test_task_ids = set(test_challenges.keys())
        submission_task_ids = set(submission.keys())
        
        missing_tasks = test_task_ids - submission_task_ids
        extra_tasks = submission_task_ids - test_task_ids
        
        if missing_tasks:
            return False, f"❌ Missing tasks: {list(missing_tasks)[:5]}"
        if extra_tasks:
            return False, f"❌ Extra tasks: {list(extra_tasks)[:5]}"
        
        print(f"✅ Task coverage: {len(submission_task_ids)} tasks (complete)")
        
        # 3. Validate each task
        errors = []
        for task_id, predictions in submission.items():
            expected_count = len(test_challenges[task_id]['test'])
            is_valid, msg = SubmissionValidator.validate_task_predictions(
                task_id, predictions, expected_count
            )
            if not is_valid:
                errors.append(msg)
                if len(errors) >= 5:  # Only show first 5 errors
                    break
        
        if errors:
            return False, f"❌ Validation errors:\n" + "\n".join(errors)
        
        print(f"✅ All predictions: Valid format")
        print("\n" + "="*70)
        print("🎉 SUBMISSION VALIDATION PASSED!")
        print("="*70 + "\n")
        
        return True, "Valid submission"
    
    @staticmethod
    def validate_and_save(submission: Dict, 
                         test_challenges: Dict,
                         output_path: str = None) -> bool:
        """Validate and save submission in one atomic operation"""
        
        if output_path is None:
            output_path = config.OUTPUT_PATH
        
        # Validate first
        is_valid, msg = SubmissionValidator.validate_submission(submission, test_challenges)
        
        if not is_valid:
            print(f"❌ VALIDATION FAILED: {msg}")
            print("❌ Submission NOT saved")
            return False
        
        # Save
        try:
            with open(output_path, 'w') as f:
                json.dump(submission, f, indent=2)
            
            file_size = os.path.getsize(output_path)
            print(f"✅ Saved to: {output_path}")
            print(f"   File size: {file_size:,} bytes")
            return True
            
        except Exception as e:
            print(f"❌ Error saving: {e}")
            return False

print("✅ Validator loaded")
print("   Catches: dict vs list, missing attempts, ragged arrays, invalid values")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: SAFE DEFAULTS & FALLBACKS
# "Valid 5% > Crashing 95%" - Never crash, always return valid
# Production-grade error recovery (Insight #6)
# ═══════════════════════════════════════════════════════════════════════════════

class SafeDefaults:
    """
    Production-grade fallback strategies.
    These NEVER throw exceptions. They ALWAYS return valid grids.
    """
    
    @staticmethod
    def copy_input(task_data: Dict, test_idx: int = 0) -> List[List[int]]:
        """Strategy 1: Copy test input (safest default)"""
        try:
            return task_data['test'][test_idx]['input']
        except:
            return [[0]]
    
    @staticmethod
    def copy_train_output(task_data: Dict, train_idx: int = 0) -> List[List[int]]:
        """Strategy 2: Copy first training output"""
        try:
            return task_data['train'][train_idx]['output']
        except:
            return SafeDefaults.copy_input(task_data)
    
    @staticmethod
    def blank_grid(task_data: Dict, test_idx: int = 0) -> List[List[int]]:
        """Strategy 3: Generate blank grid matching input size"""
        try:
            test_input = task_data['test'][test_idx]['input']
            rows = len(test_input)
            cols = len(test_input[0]) if rows > 0 else 1
            return [[0 for _ in range(cols)] for _ in range(rows)]
        except:
            return [[0]]
    
    @staticmethod
    def identity_transform(task_data: Dict, test_idx: int = 0) -> List[List[int]]:
        """Strategy 4: Identity (copy input)"""
        return SafeDefaults.copy_input(task_data, test_idx)
    
    @staticmethod
    def get_two_attempts(task_data: Dict, test_idx: int = 0) -> Tuple[List[List[int]], List[List[int]]]:
        """
        Get two safe default attempts.
        Attempt 1: Copy input
        Attempt 2: Try train output, fallback to blank
        """
        attempt_1 = SafeDefaults.copy_input(task_data, test_idx)
        
        try:
            attempt_2 = SafeDefaults.copy_train_output(task_data)
        except:
            attempt_2 = SafeDefaults.blank_grid(task_data, test_idx)
        
        return attempt_1, attempt_2
    
    @staticmethod
    def safe_execute(func, *args, fallback_func=None, **kwargs):
        """
        Execute function with automatic fallback.
        NEVER crashes. ALWAYS returns something valid.
        """
        try:
            result = func(*args, **kwargs)
            
            # Validate result is a grid
            is_valid, _ = SubmissionValidator.validate_grid(result)
            if is_valid:
                return result
            else:
                if config.VERBOSE:
                    print(f"⚠️  {func.__name__} returned invalid grid, using fallback")
                
        except Exception as e:
            if config.VERBOSE:
                print(f"⚠️  {func.__name__} failed: {str(e)[:50]}")
        
        # Use fallback
        if fallback_func:
            try:
                return fallback_func(*args, **kwargs)
            except:
                pass
        
        # Ultimate fallback
        if 'task_data' in kwargs:
            return SafeDefaults.copy_input(kwargs['task_data'])
        elif len(args) > 1 and isinstance(args[1], dict):
            return SafeDefaults.copy_input(args[1])
        else:
            return [[0]]

print("✅ Safe defaults loaded")
print("   4-tier fallback: copy_input → copy_train → blank_grid → [[0]]")
print("   Never crashes. Always returns valid.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: SUBMISSION GENERATOR
# Wire to solver with production-grade orchestration
# Enforces correct format: dict, not list! (Insight from failed submission)
# ═══════════════════════════════════════════════════════════════════════════════

class SubmissionGenerator:
    """
    Generate perfectly formatted submission.json for ARC Prize 2025.
    
    Correct format (learned from failure):
    {
      "task_id": [
        {"attempt_1": [[...]], "attempt_2": [[...]]},
        # More dicts if multiple test outputs
      ]
    }
    """
    
    def __init__(self, solver_func=None, verbose=True):
        """
        Args:
            solver_func: Function(test_input, task_data, attempt) -> grid
            verbose: Enable progress logging
        """
        self.solver_func = solver_func
        self.verbose = verbose
        self.stats = {
            'total_tasks': 0,
            'total_predictions': 0,
            'solver_successes': 0,
            'fallback_uses': 0,
            'errors': []
        }
    
    def generate_attempts(self, 
                         test_input: List[List[int]], 
                         task_data: Dict,
                         test_idx: int = 0) -> Tuple[List[List[int]], List[List[int]]]:
        """
        Generate two attempts for a single test input.
        Uses solver if available, falls back to safe defaults.
        """
        
        attempt_1 = None
        attempt_2 = None
        
        # Try solver for attempt 1
        if self.solver_func:
            attempt_1 = SafeDefaults.safe_execute(
                self.solver_func,
                test_input=test_input,
                task_data=task_data,
                attempt=1,
                fallback_func=None
            )
            
            if attempt_1 is not None and isinstance(attempt_1, list):
                self.stats['solver_successes'] += 1
        
        # Fallback for attempt 1
        if attempt_1 is None:
            attempt_1 = SafeDefaults.copy_input(task_data, test_idx)
            self.stats['fallback_uses'] += 1
        
        # Try solver for attempt 2 (with variation)
        if self.solver_func:
            attempt_2 = SafeDefaults.safe_execute(
                self.solver_func,
                test_input=test_input,
                task_data=task_data,
                attempt=2,
                fallback_func=None
            )
            
            if attempt_2 is not None and isinstance(attempt_2, list):
                self.stats['solver_successes'] += 1
        
        # Fallback for attempt 2 (different strategy)
        if attempt_2 is None:
            attempt_2 = SafeDefaults.copy_train_output(task_data)
            self.stats['fallback_uses'] += 1
        
        return attempt_1, attempt_2
    
    def generate_submission(self, 
                           test_challenges: Dict) -> Dict[str, List[Dict[str, List[List[int]]]]]:
        """
        Generate complete submission from test challenges.
        
        Returns:
            Properly formatted submission dictionary ready for validation
        """
        
        print("\n" + "="*70)
        print("🔨 GENERATING SUBMISSION")
        print("="*70)
        
        submission = {}  # DICT, not list!
        total_tasks = len(test_challenges)
        
        for task_idx, (task_id, task_data) in enumerate(test_challenges.items(), 1):
            
            # Progress update
            if self.verbose and task_idx % 20 == 0:
                pct = (task_idx / total_tasks) * 100
                elapsed = timer.elapsed() / 60
                print(f"   [{task_idx:3}/{total_tasks}] {pct:5.1f}% | "
                      f"Elapsed: {elapsed:.1f}min | "
                      f"Solver: {self.stats['solver_successes']} | "
                      f"Fallback: {self.stats['fallback_uses']}")
            
            # Check time budget (95% rule)
            if timer.should_stop():
                print(f"⏰ Approaching time limit, wrapping up...")
                # Fill remaining tasks with safe defaults
                for remaining_id in list(test_challenges.keys())[task_idx-1:]:
                    remaining_data = test_challenges[remaining_id]
                    num_tests = len(remaining_data['test'])
                    submission[remaining_id] = [
                        {
                            "attempt_1": SafeDefaults.copy_input(remaining_data, i),
                            "attempt_2": SafeDefaults.copy_train_output(remaining_data)
                        }
                        for i in range(num_tests)
                    ]
                break
            
            # Generate predictions for this task
            task_predictions = []
            num_test_outputs = len(task_data['test'])
            
            for test_idx in range(num_test_outputs):
                try:
                    test_input = task_data['test'][test_idx]['input']
                    
                    # Generate two attempts
                    attempt_1, attempt_2 = self.generate_attempts(
                        test_input, task_data, test_idx
                    )
                    
                    # Create prediction dict
                    prediction = {
                        "attempt_1": attempt_1,
                        "attempt_2": attempt_2
                    }
                    
                    # Validate before adding
                    is_valid, msg = SubmissionValidator.validate_prediction(
                        prediction,
                        context=f"Task {task_id}[{test_idx}]"
                    )
                    
                    if not is_valid:
                        print(f"⚠️  {msg}, using safe defaults")
                        attempt_1, attempt_2 = SafeDefaults.get_two_attempts(task_data, test_idx)
                        prediction = {"attempt_1": attempt_1, "attempt_2": attempt_2}
                    
                    task_predictions.append(prediction)
                    self.stats['total_predictions'] += 1
                    
                except Exception as e:
                    print(f"❌ Error on task {task_id}[{test_idx}]: {e}")
                    self.stats['errors'].append({'task_id': task_id, 'test_idx': test_idx, 'error': str(e)})
                    
                    # Emergency fallback
                    attempt_1, attempt_2 = SafeDefaults.get_two_attempts(task_data, test_idx)
                    task_predictions.append({"attempt_1": attempt_1, "attempt_2": attempt_2})
            
            # Add to submission (task_id as TOP-LEVEL KEY)
            submission[task_id] = task_predictions
            self.stats['total_tasks'] += 1
        
        print(f"\n✅ Generated {self.stats['total_tasks']} tasks, "
              f"{self.stats['total_predictions']} predictions")
        print("="*70 + "\n")
        
        return submission
    
    def print_stats(self):
        """Print generation statistics"""
        print("\n" + "="*70)
        print("📊 GENERATION STATISTICS")
        print("="*70)
        print(f"Total tasks: {self.stats['total_tasks']}")
        print(f"Total predictions: {self.stats['total_predictions']}")
        print(f"Solver successes: {self.stats['solver_successes']}")
        print(f"Fallback uses: {self.stats['fallback_uses']}")
        print(f"Errors: {len(self.stats['errors'])}")
        
        if self.stats['errors'] and config.VERBOSE:
            print("\nError summary (first 5):")
            for err in self.stats['errors'][:5]:
                print(f"  - {err['task_id']}[{err['test_idx']}]: {err['error'][:60]}")
        
        print("="*70 + "\n")

print("✅ Submission generator loaded")
print("   Format: {task_id: [{attempt_1: ..., attempt_2: ...}]}")
print("   Enforces: DICT structure, both attempts, multiple test outputs")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: SOLVER LOGIC
# Your AGI genius goes here - expand iteratively (token-efficient)
# Start simple (200 lines), test end-to-end, THEN expand (800+ lines)
# ═══════════════════════════════════════════════════════════════════════════════

def simple_solver(test_input: List[List[int]], 
                  task_data: Dict,
                  attempt: int = 1) -> List[List[int]]:
    """
    Placeholder solver - REPLACE with your actual solving logic.
    
    Args:
        test_input: The test input grid to solve
        task_data: Complete task data (includes train examples)
        attempt: Which attempt (1 or 2) - can use different strategies
    
    Returns:
        Predicted output grid (2D list of integers 0-9)
    
    TODO: Expand with insights from 48hr collaboration:
    - Phase 1: Pattern matching (flip, rotate, color swap)
    - Phase 2: Object detection (connected components)
    - Phase 3: Ensemble methods (tank/dps/healer/pug specialists)
    - Phase 4: Meta-cognition (lambda dictionary cognitive modes)
    - Phase 5: Championship polish (85%+ accuracy)
    """
    
    # For now: simple identity transform (baseline)
    # This ensures end-to-end pipeline works
    return test_input

# ═══════════════════════════════════════════════════════════════════════════════
# EXTENSION POINTS (expand as you iterate)
# ═══════════════════════════════════════════════════════════════════════════════

# def pattern_matcher_solver(test_input, task_data, attempt):
#     """Phase 1: Detect simple transformations"""
#     # TODO: Implement flip, rotate, color swap
#     pass

# def object_detection_solver(test_input, task_data, attempt):
#     """Phase 2: Object-based reasoning"""
#     # TODO: Connected components, bounding boxes
#     pass

# def ensemble_solver(test_input, task_data, attempt):
#     """Phase 3: Coordinate multiple specialists (Insight #7)"""
#     # TODO: Tank/DPS/Healer/PUG ensemble
#     pass

# def meta_cognitive_solver(test_input, task_data, attempt):
#     """Phase 4: Self-reflective reasoning (Insight #8)"""
#     # TODO: Lambda dictionary cognitive modes
#     pass

print("✅ Solver logic loaded (placeholder - expand with your AGI)")
print("   Current: Identity transform (baseline)")
print("   Expand to: Pattern matching → Objects → Ensemble → Meta-cognition")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: EXECUTION PIPELINE
# Orchestration: Load → Solve → Generate → Validate → Save
# Implements 95% rule, progress tracking, production monitoring
# ═══════════════════════════════════════════════════════════════════════════════

def main():
    """
    Main execution pipeline.
    Follows production-first, 95% rule principles.
    """
    
    print("\n" + "="*70)
    print("🐋 SUBTLEGENIUS V1 - ARC PRIZE 2025 SUBMISSION SYSTEM")
    print("="*70)
    print(f"⏰ Start time: {datetime.now().strftime('%H:%M:%S')}")
    print(f"💾 Output: {config.OUTPUT_PATH}")
    print(f"🕐 Time budget: {config.EFFECTIVE_TIME/3600:.1f}h (95% of 12h)")
    print("="*70 + "\n")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # STEP 1: Load test data
    # ═══════════════════════════════════════════════════════════════════════════
    print("📂 Loading test challenges...")
    try:
        with open(config.INPUT_PATH, 'r') as f:
            test_challenges = json.load(f)
        print(f"✅ Loaded {len(test_challenges)} tasks\n")
    except Exception as e:
        print(f"❌ Failed to load test data: {e}")
        print(f"   Path: {config.INPUT_PATH}")
        return
    
    # ═══════════════════════════════════════════════════════════════════════════
    # STEP 2: Generate submission
    # ═══════════════════════════════════════════════════════════════════════════
    generator = SubmissionGenerator(
        solver_func=simple_solver,  # REPLACE with your solver
        verbose=config.VERBOSE
    )
    
    submission = generator.generate_submission(test_challenges)
    generator.print_stats()
    
    # ═══════════════════════════════════════════════════════════════════════════
    # STEP 3: Validate and save submission
    # ═══════════════════════════════════════════════════════════════════════════
    if config.ENABLE_VALIDATION:
        is_valid = SubmissionValidator.validate_and_save(
            submission,
            test_challenges,
            config.OUTPUT_PATH
        )
        
        if not is_valid:
            print("\n" + "="*70)
            print("❌ CRITICAL: Submission validation failed!")
            print("❌ Fix errors before submitting to Kaggle!")
            print("="*70 + "\n")
            return
    else:
        # Save without validation (not recommended)
        with open(config.OUTPUT_PATH, 'w') as f:
            json.dump(submission, f, indent=2)
        print(f"✅ Saved (without validation): {config.OUTPUT_PATH}")
    
    # ═══════════════════════════════════════════════════════════════════════════
    # STEP 4: Final summary
    # ═══════════════════════════════════════════════════════════════════════════
    elapsed = timer.elapsed()
    print("\n" + "="*70)
    print("🎉 PIPELINE COMPLETE")
    print("="*70)
    print(f"⏱️  Total time: {elapsed:.1f}s ({elapsed/60:.1f}min)")
    print(f"📊 Time used: {timer.progress_pct():.1f}% of budget")
    print(f"📁 Submission: {config.OUTPUT_PATH}")
    
    if os.path.exists(config.OUTPUT_PATH):
        file_size = os.path.getsize(config.OUTPUT_PATH)
        print(f"📏 File size: {file_size:,} bytes")
    
    print("="*70)
    print("\n🚀 Ready for Kaggle submission!")
    print("   Remember:")
    print("   1. Disable internet access")
    print("   2. Open source before getting official scores")
    print("   3. Save logs for post-submission analysis")
    print("="*70 + "\n")
    
    # Cleanup
    gc.collect()

# ═══════════════════════════════════════════════════════════════════════════════
# RUN THE PIPELINE
# ═══════════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    main()

print("✅ Execution pipeline ready")
print("   Run main() to generate submission.json")